# TrajectoryCollection aggregation (flow maps)

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/movingpandas/movingpandas/main?filepath=tutorials/3-generalization-and-aggregation.ipynb)

**<p style="color:#e31883">This notebook demonstrates the current development version of MovingPandas.</p>**

For tutorials using the latest release visit https://github.com/movingpandas/movingpandas-examples.


In [ ]:
import urllib
import os
import numpy as np
import pandas as pd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon, MultiPoint
from datetime import datetime, timedelta
from holoviews import opts, dim

import sys

sys.path.append("..")
import movingpandas as mpd

mpd.show_versions()

import warnings

warnings.simplefilter("ignore")

## Ship movements (AIS data)

In [ ]:
df = read_file("data/demodata_ais.gpkg")
df["t"] = pd.to_datetime(df["Timestamp"], format="%d/%m/%Y %H:%M:%S")
df = df[df.SOG > 0]
len(df)

In [ ]:
MIN_LENGTH = 100  # meters
traj_collection = mpd.TrajectoryCollection(df, "MMSI", min_length=MIN_LENGTH, t="t")
print("Finished creating {} trajectories".format(len(traj_collection)))

In [ ]:
trips = mpd.ObservationGapSplitter(traj_collection).split(gap=timedelta(minutes=5))
print(
    "Extracted {} individual trips from {} continuous vessel tracks".format(
        len(trips), len(traj_collection)
    )
)

Generalizing the trip trajectories significantly speeds up the following aggregation step.

In [ ]:
%%time
generalized = mpd.MinDistanceGeneralizer(trips).generalize(tolerance=100)

In [ ]:
%%time
aggregator = mpd.TrajectoryCollectionAggregator(
    generalized,
    max_distance=1000,
    min_distance=100,
    min_stop_duration=timedelta(minutes=5),
)

In [ ]:
from hvplot import pandas

pts = aggregator.get_significant_points_gdf()
clusters = aggregator.get_clusters_gdf()
(
    pts.hvplot(geo=True, tiles="OSM", frame_width=800)
    * clusters.hvplot(geo=True, color="red")
)

In [ ]:
flows = aggregator.get_flows_gdf()

In [ ]:
(  # trips.hvplot(color='gray')  *
    flows.hvplot(
        geo=True,
        hover_cols=["weight"],
        line_width=dim("weight") * 0.3,
        alpha=0.5,
        color="#1f77b3",
        tiles="OSM",
    )
    * clusters.hvplot(geo=True, color="red", size="n")
)

### Comparison of generalized vs. original trajectories

In [ ]:
%%time
aggregator_original = mpd.TrajectoryCollectionAggregator(
    trips, max_distance=1000, min_distance=100, min_stop_duration=timedelta(minutes=5)
)

In [ ]:
(
    aggregator_original.get_flows_gdf().hvplot(
        title="Original",
        geo=True,
        tiles="OSM",
        hover_cols=["weight"],
        line_width=dim("weight") * 0.3,
        alpha=0.5,
        color="#1f77b3",
        frame_height=400,
        frame_width=400,
    )
    * aggregator_original.get_clusters_gdf().hvplot(geo=True, color="red", size="n")
    + flows.hvplot(
        title="Generalized",
        geo=True,
        tiles="OSM",
        hover_cols=["weight"],
        line_width=dim("weight") * 0.3,
        alpha=0.5,
        color="#1f77b3",
        frame_height=400,
        frame_width=400,
    )
    * clusters.hvplot(geo=True, color="red", size="n")
)

## Bird migration data

In [ ]:
df = read_file("data/demodata_gulls.gpkg")
len(df)

In [ ]:
traj_collection = mpd.TrajectoryCollection(
    df, "individual-local-identifier", min_length=MIN_LENGTH, t="timestamp"
)
print("Finished creating {} trajectories".format(len(traj_collection)))

In [ ]:
trips = mpd.TemporalSplitter(traj_collection).split(mode="month")
print(
    "Extracted {} individual trips from {} continuous tracks".format(
        len(trips), len(traj_collection)
    )
)

In [ ]:
generalized = mpd.MinTimeDeltaGeneralizer(trips).generalize(tolerance=timedelta(days=1))

In [ ]:
%%time
aggregator = mpd.TrajectoryCollectionAggregator(
    generalized,
    max_distance=1000000,
    min_distance=100000,
    min_stop_duration=timedelta(minutes=5),
)

In [ ]:
flows = aggregator.get_flows_gdf()
clusters = aggregator.get_clusters_gdf()

(
    flows.hvplot(
        geo=True,
        hover_cols=["weight"],
        line_width=dim("weight") * 0.8,
        alpha=0.5,
        color="#1f77b3",
        tiles="OSM",
        frame_height=600,
        frame_width=400,
    )
    * clusters.hvplot(geo=True, color="red", size="n")
)

In [ ]:
%%time
aggregator_original = mpd.TrajectoryCollectionAggregator(
    trips,
    max_distance=1000000,
    min_distance=100000,
    min_stop_duration=timedelta(minutes=5),
)

In [ ]:
(
    aggregator_original.get_flows_gdf().hvplot(
        title="Original",
        geo=True,
        tiles="OSM",
        hover_cols=["weight"],
        line_width=dim("weight") * 0.8,
        alpha=0.5,
        color="#1f77b3",
        frame_height=600,
        frame_width=400,
    )
    * aggregator_original.get_clusters_gdf().hvplot(geo=True, color="red", size="n")
    + flows.hvplot(
        title="Generalized",
        geo=True,
        tiles="OSM",
        hover_cols=["weight"],
        line_width=dim("weight") * 0.8,
        alpha=0.5,
        color="#1f77b3",
        frame_height=600,
        frame_width=400,
    )
    * clusters.hvplot(geo=True, color="red", size="n")
)

## Continue exploring MovingPandas

1. [Getting started](1-getting-started.ipynb)
1. [Handling trajectory data files (reading & writing)](2-reading-data-from-files.ipynb)
1. [TrajectoryCollection aggregation (flow maps)](3-generalization-and-aggregation.ipynb)
1. [Stop detection](4-stop-detection.ipynb)
1. [Working with local coordinates](5-local-coordinates.ipynb)
1. [Computing trajectory metrics](6-trajectory-metrics.ipynb)
1. [Multithreading](7-multithreading.ipynb)
1. [OGC Moving Features](8-ogc-moving-features.ipynb)